In [56]:
import urllib.request
import re
from bs4 import BeautifulSoup
from lxml import html
import pandas as pd
import json
import requests
import json

In [57]:
def strip_price(price):
    rtn = ''
    aGroup = '0123456789.'
    for i in price:
        if i in aGroup:
            rtn += i
    return rtn

In [58]:
def luckyGunner_shotgun_helper(url):
    url+='?limit-all'
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 \
        (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'
    }
        
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "lxml")
  
    products = soup.find_all('ol', class_='products-list')
    caliber = soup.find('div', class_='category-title')

    list_of_dicts = []
    for detail in products:
        

        d = detail.find_all('li', class_='item')
        lastItem = detail.find('li', class_='item last')
        
        # scrape prices
        for price in d:
            aDict = {'name': '', 'price': '', 'link': '', 'caliber': ''}
            if_available = (price.find('p', class_='availability'))
    
            if 'Out of stock' not in str(if_available.text):
                aDict['name'] = price.find('span').text
                dLink = price.find('a')
                aDict['link'] = dLink['href']
                aDict['caliber'] = caliber.text.replace('\n', '')
                
                if "Special Price" in str(price):
                    productPrice = price.find('p', class_='special-price')
                    aDict['price'] = strip_price(productPrice.text)
                else:
                    productPrice = price.find('span', class_='regular-price')
                    aDict['price'] = strip_price(productPrice.text)
            else:
                aDict['price'] = 'out of stock'
            list_of_dicts.append(aDict)
    return list_of_dicts
    

        
        

        
    

        
 


In [59]:
def luckyGunner_shotgun_scrape(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 \
        (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'
    }
        
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "lxml")
    productLinks = soup.find_all('td')
    calibers  = []
    detailLink = []
    ammo_data = {"type":"shotgun", "ammo_results": []}
    for i in productLinks:
        scrapedLink = i.find('a')
        detailLink.append(scrapedLink["href"])
        
        results = luckyGunner_shotgun_helper(scrapedLink["href"])
        if len(results) != 0:
            ammo_data['ammo_results'].append(results)
        else:
            out_of_stock = scrapedLink['title']
            ammo_data['ammo_results'].append(out_of_stock + ': out of stock')
            
    return ammo_data

# TODO: if resuly is [], then its out of stock, finish getting json for this file


url = "https://www.luckygunner.com/shotgun"
ammo_data = luckyGunner_shotgun_scrape(url)

In [60]:
from pathlib import Path
import os, shutil, glob
rel = "jsonFiles/luckyGunner/ammo"
cwd = Path.cwd()
currentPath = str(Path.cwd())+""
aCount = 0
savePath = str(cwd).replace("pyScripts/luckyGunner", rel)

with open(os.path.join(savePath, f"LGShotgunAmmo.json"), "w") as fp:
    json.dump(ammo_data, fp)


rel = 'crispy-computing-app/crispy-computing-app/jsonFiles/luckyGunner/ammo'
cwd = Path.cwd()
currentPath = str(Path.cwd())+''
savePath = str(cwd).replace("pyScripts/luckyGunner", rel)
index = 0
with open(os.path.join(savePath, f"LGShotgunAmmo.json"), "w") as fp:
    json.dump(ammo_data, fp)